# Blue Brain BioExplorer
![](../../../bioexplorer_banner.png)

# High Glucose Scenario

## Connect to BioExplorer backend

In [ ]:
from bioexplorer import BioExplorer, Protein, Surfactant, Membrane, Cell, Sugar, \
                        Volume, AnimationParams, Vector2, Vector3, Quaternion
import nglview

hostname = 'localhost:5000'
resource_folder = '../../../../tests/test_files/'

be = BioExplorer(hostname)
core = be.core_api()
version = be.version()
be.reset_scene()
print('Version: ' + str(version))

export_to_cache = False

scenario = 'high_glucose'

In [ ]:
'''Accelerate loading by not showing models as they are loaded'''
status = be.set_general_settings(
    model_visibility_on_creation=False,
    v1_compatibility=True
)
status = core.set_renderer(samples_per_pixel=1, subsampling=1, max_accum_frames=1)

## Global parameters

In [ ]:
# Scene
scene_size = Vector3(800.0, 800.0, 800.0)

# Proteins
protein_radius_multiplier = 1.0
protein_representation = be.REPRESENTATION_ATOMS_AND_STICKS
protein_load_hydrogen = False

# Glycans
add_glycans = True
glycan_radius_multiplier = 1.0
glycan_representation = be.REPRESENTATION_ATOMS_AND_STICKS

# Viruses
nb_protein_s = 62
nb_protein_e = 42
nb_protein_m = 50
add_rna = False

# Immune system
nb_glucoses = 360000
nb_lactoferrins = 50
nb_defensins = 100
nb_defensins_on_virus = 2

# Cell
cell_name = 'Cell'
cell_position = Vector3(4.5, -186.0, 7.0)
cell_height = scene_size.y / 10.0
cell_nb_receptors = 100

## Resources

In [ ]:
cache_folder = resource_folder + 'caches/' + version + '/'
image_folder = resource_folder + 'images/'
pdb_folder = resource_folder + 'pdb/'
rna_folder = resource_folder + 'rna/'
obj_folder = resource_folder + 'obj/'
glycan_folder = pdb_folder + 'glycans/'
membrane_folder = pdb_folder + 'membrane/'

complex_paths = [glycan_folder + 'complex/33.pdb', glycan_folder + 'complex/34.pdb',
                 glycan_folder + 'complex/35.pdb',glycan_folder + 'complex/36.pdb']
high_mannose_paths = [glycan_folder + 'high-mannose/1.pdb', 
                      glycan_folder + 'high-mannose/2.pdb',
                      glycan_folder + 'high-mannose/3.pdb',
                      glycan_folder + 'high-mannose/4.pdb']
hybrid_paths = [glycan_folder + 'hybrid/24.pdb']
o_glycan_paths = [glycan_folder + 'o-glycan/12.pdb']

glucose_path = pdb_folder + 'glucose.pdb'
lactoferrin_path=pdb_folder + 'immune/1b0l.pdb'
defensin_path = pdb_folder + 'immune/1ijv.pdb'

surfactant_head_source = pdb_folder + 'surfactant/1pw9.pdb'
surfactant_branch_source = pdb_folder + 'surfactant/1k6f.pdb'

## SARS-COV-2 Coronaviruses

In [ ]:
nglview.show_file(pdb_folder + 'sars-cov-2-v1.pdb')

In [ ]:
nglview.show_file(pdb_folder + '6vyb.pdb')

In [ ]:
nglview.show_file(pdb_folder + 'QHD43419a.pdb')

In [ ]:
nglview.show_file(pdb_folder + 'QHD43418a.pdb')

In [ ]:
be.add_sars_cov_2(
    name='sars-cov-2 1', resource_folder=resource_folder,
    representation=protein_representation,
    position=Vector3(-337.3, -92.3, -99.2), add_glycans=add_glycans)
be.add_sars_cov_2(
    name='sars-cov-2 2', resource_folder=resource_folder,
    representation=protein_representation,
    position=Vector3(-74.9, -97.1, 228.8), add_glycans=add_glycans)
be.add_sars_cov_2(
    name='sars-cov-2 3', resource_folder=resource_folder,
    representation=protein_representation,
    position=Vector3(187.5, -110.4, 51.2), add_glycans=add_glycans)
be.add_sars_cov_2(
    name='sars-cov-2 4', resource_folder=resource_folder,
    representation=protein_representation,
    position=Vector3(4.5, 100.0, 7.5),
    rotation=Quaternion(0.0, 0.0, 0.0, 1.0), 
    add_glycans=add_glycans)
be.add_sars_cov_2(
    name='sars-cov-2 5', resource_folder=resource_folder,
    representation=protein_representation,
    position=Vector3(73.9, -117.1, -190.4), add_glycans=add_glycans)
be.add_sars_cov_2(
    name='sars-cov-2 6', resource_folder=resource_folder,
    representation=protein_representation,
    position=Vector3(211.5, -104.9, 339.2), add_glycans=add_glycans)

In [ ]:
if export_to_cache:
    '''Show loaded models'''
    status = be.set_models_visibility(True)
    '''Set materials'''
    be.apply_default_color_scheme(shading_mode=be.SHADING_MODE_BASIC)
    '''Export to cache file'''
    cache_filename = cache_folder + scenario + '_viruses.bioexplorer'
    if add_rna:
        cache_filename = cache_folder + scenario + '_open_viruses.bioexplorer'
    print(cache_filename)
    be.export_to_file(cache_filename)

## Host cell

In [ ]:
def add_cell(
        name, size, nb_receptors,
        position=Vector3()):
    
    seed = 10
    ace2_receptor = Protein(
        name=name + '_' + be.NAME_RECEPTOR,
        source=pdb_folder + '6m18.pdb', occurrences=nb_receptors,
        transmembrane_params=Vector2(-6.0, 5.0),
        animation_params=AnimationParams(seed, 1, 0.025, 2, 0.025))
    
    membrane = Membrane(
        lipid_sources=[
            membrane_folder + 'segA.pdb',
            membrane_folder + 'segB.pdb',
            membrane_folder + 'segC.pdb',
            membrane_folder + 'segD.pdb'
        ],
        animation_params=AnimationParams(seed, 2, 0.025, 3, 1.0)
    )

    cell = Cell(
        name=name,
        shape=be.ASSEMBLY_SHAPE_SINUSOID, 
        shape_params=size,
        membrane=membrane, proteins=[ace2_receptor],
    )
    
    be.add_cell(
        cell=cell, position=position,
        representation=protein_representation)

    if nb_receptors != 0 and add_glycans:
        be.add_multiple_glycans(
            representation=glycan_representation, assembly_name=name, 
            glycan_type=be.NAME_GLYCAN_COMPLEX,
            protein_name=be.NAME_RECEPTOR, paths=complex_paths, 
            indices=[53, 90, 103, 322, 432, 690])
        be.add_multiple_glycans(
            representation=glycan_representation, assembly_name=name,
            glycan_type=be.NAME_GLYCAN_HYBRID,
            protein_name=be.NAME_RECEPTOR, paths=hybrid_paths, 
            indices=[546])

        indices = [[155, Quaternion(0.707, 0.0, 0.707, 0.0)],
                   [730, Quaternion(0.707, 0.0, 0.707, 0.0)]]
        for index in indices:
            o_glycan_name = name + '_' + be.NAME_GLYCAN_O_GLYCAN + '_' + str(index[0])
            o_glycan = Sugar(
                assembly_name=name, name=o_glycan_name, source=o_glycan_paths[0],
                protein_name=name + '_' + be.NAME_RECEPTOR, representation=glycan_representation,
                chain_ids=[2, 4], site_indices=[index[0]], rotation=index[1])
            be.add_sugar(o_glycan)

In [ ]:
cell_position = Vector3(4.5, -186.0, 7.0)
add_cell(
    name='Cell',
    nb_receptors=cell_nb_receptors,
    size=Vector3(scene_size.x * 2.0, scene_size.y / 10.0, scene_size.z * 2.0),
    position=cell_position)

In [ ]:
if export_to_cache:
    '''Show loaded models'''
    status = be.set_models_visibility(True)
    '''Materials'''
    be.apply_default_color_scheme(shading_mode=be.SHADING_MODE_BASIC)
    be.set_protein_color_scheme(
        cell_name, cell_name + '_' + be.NAME_RECEPTOR,
        be.COLOR_SCHEME_CHAINS, 'OrRd_r', 7)
    '''Export to cache file'''
    cache_filename = cache_folder + 'cell.bioexplorer'
    print(cache_filename)
    be.export_to_file(cache_filename)

## Immune system

### Surfactant A and D

In [ ]:
nglview.show_file(surfactant_head_source)

In [ ]:
def add_surfactant_d(name, position, animation_params):
    surfactant_d = Surfactant(
        name=name, surfactant_protein=be.SURFACTANT_PROTEIN_D, 
        head_source=surfactant_head_source,
        branch_source=surfactant_branch_source)
    be.add_surfactant(
        surfactant=surfactant_d, representation=protein_representation, 
        position=position, animation_params=animation_params)

add_surfactant_d(
    name='Surfactant-D 1',
    position=Vector3(74.0, 24.0, -45.0),
    animation_params=AnimationParams(seed=1))
add_surfactant_d(
    name='Surfactant-D 2',
    position=Vector3(104.0, 175.0, -89.0),
    animation_params=AnimationParams(seed=2))
add_surfactant_d(
    name='Surfactant-D 3',
    position=Vector3(-260.0, 50.0, 0.0),
    animation_params=AnimationParams(seed=6))

In [ ]:
def add_surfactant_a(name, position, animation_params):
    surfactant_a = Surfactant(
        name=name, surfactant_protein=be.SURFACTANT_PROTEIN_A, 
        head_source=surfactant_head_source,
        branch_source=surfactant_branch_source)
    be.add_surfactant(
        surfactant=surfactant_a, representation=protein_representation, 
        position=position, animation_params=animation_params)

add_surfactant_a(
    name='Surfactant-A 1',
    position=Vector3(-100.0, 150.0, 0.0),
    animation_params=AnimationParams(seed=2))

In [ ]:
# Add glucose to surfactants
def add_glucose_to_surfactant_head(name):
    for index in [321, 323]:
        glucose_name = name + '_' + be.NAME_GLUCOSE + '_' + str(index)
        glucose = Sugar(
            assembly_name=name, name=glucose_name, source=glucose_path,
            protein_name=name + '_' + be.NAME_SURFACTANT_HEAD, 
            representation=glycan_representation, site_indices=[index])
        be.add_sugar(glucose)
add_glucose_to_surfactant_head('Surfactant-D 1')
add_glucose_to_surfactant_head('Surfactant-D 2')
add_glucose_to_surfactant_head('Surfactant-D 3')
add_glucose_to_surfactant_head('Surfactant-A 1')

### Glucose

In [ ]:
nglview.show_file(glucose_path)

In [ ]:
glucose = Protein(
    name=be.NAME_GLUCOSE,
    source=glucose_path, load_non_polymer_chemicals=True, 
    occurrences=nb_glucoses, animation_params=AnimationParams(0, 1, 2.0, 2, 1.0))
volume = Volume(
    name=be.NAME_GLUCOSE,
    shape=be.ASSEMBLY_SHAPE_CUBE, shape_params=scene_size,
    protein=glucose)
status = be.add_volume(
    volume=volume, representation=protein_representation,
    position=Vector3(0.0, scene_size.y / 2.0 - 200.0, 0.0))

### Lactoferrins

In [ ]:
nglview.show_file(lactoferrin_path)

In [ ]:
lactoferrin = Protein(
    name=be.NAME_LACTOFERRIN,
    source=lactoferrin_path, load_non_polymer_chemicals=True,
    occurrences=nb_lactoferrins, animation_params=AnimationParams(2, 2, 2.0, 3, 1.0))
lactoferrins_volume = Volume(
    name=be.NAME_LACTOFERRIN,
    shape=be.ASSEMBLY_SHAPE_CUBE, shape_params=scene_size,
    protein=lactoferrin)
status = be.add_volume(
    volume=lactoferrins_volume, representation=protein_representation,
    position=Vector3(0.0, scene_size.y / 2.0 - 200.0, 0.0))

### Defensins

In [ ]:
nglview.show_file(defensin_path)

In [ ]:
defensin = Protein(
    name=be.NAME_DEFENSIN,
    source=defensin_path, load_non_polymer_chemicals=True, 
    occurrences=nb_defensins, animation_params=AnimationParams(3, 3, 2.0, 4, 1.0))
defensins_volume = Volume(
    name=be.NAME_DEFENSIN, shape=be.ASSEMBLY_SHAPE_CUBE, shape_params=scene_size,
    protein=defensin)
status = be.add_volume(
    volume=defensins_volume, representation=protein_representation,
    position=Vector3(0.0, scene_size.y / 2.0 - 200.0, 0.0))

### Materials

In [ ]:
be.apply_default_color_scheme(
    shading_mode=be.SHADING_MODE_DIFFUSE,
    specular_exponent=50.0)

In [ ]:
'''Show loaded models'''
status = be.set_models_visibility(True)

In [ ]:
if export_to_cache:
    '''Export to cache file'''
    cache_filename = cache_folder + scenario + '_immune.bioexplorer'
    print(cache_filename)
    be.export_to_file(cache_filename)

## Export full scene to cache file

In [ ]:
if export_to_cache:
    '''Export to cache file'''
    cache_filename = cache_folder + scenario + '_scenario.bioexplorer'
    print(cache_filename)
    be.export_to_file(cache_filename)

## Rendering settings

In [32]:
status = be.set_rendering_quality(be.RENDERING_QUALITY_HIGH)